In [22]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

In [5]:
# to get pdf texts
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)

        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [6]:
# to get text chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    chunks = text_splitter.split_text(text)
    return chunks

In [7]:
# make a vector store if you want through api
def get_vectorstore(text_chunks):
    
    inference_api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")
    embeddings = HuggingFaceInferenceAPIEmbeddings(api_key = inference_api_key, model_name="hkunlp/instructor-xl")

    vectorstore = FAISS.from_texts(text_chunks, embeddings)
    return vectorstore

In [8]:
# # without using api
# from langchain_community.embeddings import HuggingFaceEmbeddings

# def get_vectorstore(text_chunks):
    
#     embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-xl")
#     vectorstore = FAISS.from_texts(text_chunks, embeddings)
#     return vectorstore

In [34]:
def get_conversation_chain(vectorstore):
    
    llm = HuggingFaceHub( repo_id="mistralai/Mistral-7B-Instruct-v0.2", model_kwargs={"temperature": 0.5, "max_length": 64, "max_new_tokens":512})

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, memory_size=500)
    
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type='stuff',
        retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),
        memory=memory
    )
    return conversation_chain

In [35]:
# Define chat function
def conversation_chat(conversation_chain, query, history):

    result = conversation_chain({"question": query, "chat_history": history})
    history.append((query, result["answer"]))
    return result["answer"]

In [36]:

def main(user_input):

    history = []

    # Load PDF documents
    pdf_docs = ["check1.pdf"]  
    
    # Get PDF text
    raw_text = get_pdf_text(pdf_docs)

    # Get text chunks
    text_chunks = get_text_chunks(raw_text)

    # Create vector store
    vectorstore = get_vectorstore(text_chunks)

    # creating the chain
    conversation_chain = get_conversation_chain(vectorstore)

    output = conversation_chat(conversation_chain ,user_input, history)

    return output


In [72]:
user_input = "What sectors saw the fastest job growth according to the fact sheet?"
output = main(user_input)

In [71]:
import re
match = re.findall(r'\nHelpful Answer: (.*)', output)

for item in match:
    print(item)

Renewable energy jobs refer to employment opportunities within the renewable energy sector, which includes industries such as wind, solar, hydroelectric, geothermal, and biomass energy. These jobs can range from manufacturing and installing renewable energy systems, to operating and maintaining them, to conducting research and development, and to providing policy and regulatory support. According to the Environmental Entrepreneurs and the U.S. Department of Energy, the clean energy industry, which includes renewable energy, employed over 4 million people in the United States as of 2023. The top states for renewable energy employment were California, Texas, Florida, New York, and Iowa. The electric power generation, transmission, and distribution sector, which includes renewable energy, employed over 600,000 people in the United States as of 2023.
